In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import missingno as msno

In [ ]:
train_df = pd.read_csv('../datasets/train.csv')
'''Removing the 'id' column is a common preprocessing step in machine learning workflows.
id's do not carry predictive information and could interfere with model training if left in the dataset.
 By dropping this column from both training and test sets, you ensure that only relevant features are used for modeling'''
train_df.drop('id',inplace=True,axis=1)
print('Columns:', list(train_df.columns))
print('Info:')
train_df.info()
train_df.head()

In [ ]:
# check pseudonull values and uniqueness faults
print("\nUnique values in each column (nan is not a value):")
for col in train_df.columns:
    print(f"{col}: {train_df[col].nunique()} unique values")
    if col != "id":
        try:
            unique_vals = train_df[col].unique()
            # Convert to string to handle mixed types, then sort
            unique_vals_str = [str(val) for val in unique_vals]
            print(f"  Values: {sorted(unique_vals_str)}")
        except Exception as e:
            print(f"  Values: {list(train_df[col].unique())}")
    print()

"brand" column has no pseudonull values and unique values are OK
"model" column has no pseudonull values and unique values are OK
"model_year" column has no pseudonull values and unique values are OK
"mileage" column has no pseudonull values and unique values are OK
"fuel_type" column has pseudonull values "-"
"engine" column has pseudonull values "-"
"transmission" column has pseudonull values "-"
"ext_col" column has pseudonull values "-"
"int_col" column has pseudonull values "-"
"accident" column has no pseudonull values and unique values are OK
"clean_title" column has no pseudonull values and unique values are OK
"price" column has no pseudonull values and unique values are OK

In [ ]:
# replace "-" values by nan
df_clean = train_df.copy()
missing_values = ['-', '—', '–', '−']
df_clean = df_clean.replace(missing_values, np.nan)
print('Info after replacing "-" by nan:')
df_clean.info()

In [ ]:
msno.matrix(df_clean, figsize=(12, 8))
plt.title('Missing Values distribution')
plt.show()

In [ ]:
#check for duplicate rows
duplicates = df_clean[df_clean.duplicated(keep=False)]#df_clean.duplicated(keep=False) returns a boolean Series indicating if a row is duplicated in the  DataFrame
duplicates.shape

There are no duplicate rows in our dataset

In [ ]:
# delete any row with nan value
df_clean.dropna(inplace=True)
print('Info after deleting nan rows')
df_clean.info()

In [ ]:
from datetime import datetime

def feature_engineering(df_clean):
    current_year = datetime.now().year

    df_clean['age'] = current_year - df_clean['model_year']
    df_clean['milage_per_year'] = df_clean['milage']/df_clean['age']

    def extract_horsepower(engine):
        try:
            return float(engine.split('HP')[0])
        except:
            return None

    def extract_engine_size(engine):
        try:
            return float(engine.split(' ')[1].replace('L', ''))
        except:
            return None

    df_clean['horsepower'] = df_clean['engine'].apply(extract_horsepower)# when splitting fails nan values are assigned
    df_clean['engine_size'] = df_clean['engine'].apply(extract_engine_size)
    df_clean['power_to_weight_ratio'] = df_clean['horsepower']/df_clean['engine_size']

    luxury_brands =  ['Mercedes-Benz', 'BMW', 'Audi', 'Porsche', 'Land', 
                    'Lexus', 'Jaguar', 'Bentley', 'Maserati', 'Lamborghini', 
                    'Rolls-Royce', 'Ferrari', 'McLaren', 'Aston', 'Maybach']
    df_clean['Is_Luxury_Brand'] = df_clean['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

    df_clean['Accident_Impact'] = df_clean.apply(lambda x: 1 if x['accident'] == 1 and x['clean_title'] == 0 else 0, axis=1)
    # Nan values are generated throug feature enginereeing. Drop rows with any NaN values after feature extraction
    df_clean = df_clean.dropna()
    return df_clean


In [ ]:
df_clean = feature_engineering(df_clean)
print("\nNaNs in df_clean after feature engineering:", df_clean.isnull().sum())
df_clean.to_csv('../datasets/train_cleaned.csv')

## Metrics
Now that we have cleaned our dataset, let´s get its metrics

## Univariate Analysis

Now we'll perform a comprehensive univariate analysis to understand the distribution and characteristics of each variable in our cleaned dataset.


In [ ]:
# UNIVARIATE VISUALIZATIONS
print("\n" + "=" * 60)
print("UNIVARIATE ANALYSIS - VISUALIZATIONS")
print("=" * 60)

# Set up the plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 30)  # taller for 12 plots

# 1. NUMERIC VARIABLES PLOTS
numeric_columns = df_clean.select_dtypes(include=['number']).columns
n_cols = 3
n_rows = 6
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 30))
fig.suptitle('Univariate Analysis - Numeric Variables', fontsize=16, fontweight='bold')

for idx, col in enumerate(numeric_columns):
    row = idx // n_cols
    col_idx = idx % n_cols
    # Histogram in the upper half of each cell
    if col == 'Accident_Impact':
        axes[row, col_idx].hist(df_clean[col].dropna(), bins=[-0.5,0.5,1.5], alpha=0.7, color='skyblue', edgecolor='black', rwidth=0.8)
        axes[row, col_idx].set_xlim(-0.1, 1.1)
        axes[row, col_idx].set_xticks([0, 1])
    else:
        axes[row, col_idx].hist(df_clean[col].dropna(), bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    axes[row, col_idx].set_title(f'{col} - Distribution', fontweight='bold')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')
    axes[row, col_idx].grid(True, alpha=0.3)

# Hide any unused subplots (remove empty histograms)
for idx in range(len(numeric_columns), n_rows * n_cols):
    row = idx // n_cols
    col_idx = idx % n_cols
    fig.delaxes(axes[row, col_idx])

plt.tight_layout(rect=[0, 0.5, 1, 0.97])
plt.show()

# Boxplots for numeric columns
fig2, axes2 = plt.subplots(n_rows, n_cols, figsize=(18, 30))
fig2.suptitle('Univariate Analysis - Numeric Variables (Boxplots)', fontsize=16, fontweight='bold')

for idx, col in enumerate(numeric_columns):
    row = idx // n_cols
    col_idx = idx % n_cols
    axes2[row, col_idx].boxplot(df_clean[col].dropna(), patch_artist=True, boxprops=dict(facecolor='lightgreen', alpha=0.7))
    axes2[row, col_idx].set_title(f'{col} - Box Plot', fontweight='bold')
    axes2[row, col_idx].set_ylabel(col)
    axes2[row, col_idx].grid(True, alpha=0.3)

# Hide any unused subplots in boxplots
for idx in range(len(numeric_columns), n_rows * n_cols):
    row = idx // n_cols
    col_idx = idx % n_cols
    fig2.delaxes(axes2[row, col_idx])

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()

In [ ]:
# CATEGORICAL VARIABLES PLOTS
categorical_columns = ['brand', 'model', 'fuel_type', 'engine', 'transmission', 'ext_col', 'int_col', 'accident', 'clean_title']

# Split into 3 groups of 3 columns each (6 charts per group)
group1 = categorical_columns[:3]  # First 3 columns: brand, model, fuel_type
group2 = categorical_columns[3:6]  # Next 3 columns: engine, transmission, ext_col
group3 = categorical_columns[6:]  # Last 3 columns: int_col, accident, clean_title

# FIGURE 1: First 3 columns (6 charts)
fig1, axes1 = plt.subplots(2, 3, figsize=(18, 12))
fig1.suptitle('Categorical Variables', fontsize=16, fontweight='bold')

for i, col in enumerate(group1):
    value_counts = df_clean[col].value_counts().head(8)
    
    # Bar chart
    axes1[0, i].bar(range(len(value_counts)), value_counts.values, 
                   color='lightcoral', alpha=0.7, edgecolor='black')
    axes1[0, i].set_title(f'{col}', fontweight='bold')
    axes1[0, i].set_xlabel(col)
    axes1[0, i].set_ylabel('Count')
    axes1[0, i].tick_params(axis='x', rotation=45)
    axes1[0, i].grid(True, alpha=0.3)
    axes1[0, i].set_xticks(range(len(value_counts)))
    axes1[0, i].set_xticklabels(value_counts.index, rotation=45, ha='right')
    
    # Pie chart
    axes1[1, i].pie(value_counts.values, labels=value_counts.index, 
                   autopct='%1.1f%%', startangle=90)
    axes1[1, i].set_title(f'{col}', fontweight='bold')

plt.tight_layout()
plt.show()

# FIGURE 2: Next 3 columns (6 charts)
fig2, axes2 = plt.subplots(2, 3, figsize=(18, 12))
#fig2.suptitle('Categorical Variables - Group 2 (Bar + Pie Charts)', fontsize=16, fontweight='bold')

for i, col in enumerate(group2):
    value_counts = df_clean[col].value_counts().head(8)
    
    # Bar chart
    axes2[0, i].bar(range(len(value_counts)), value_counts.values, 
                   color='lightblue', alpha=0.7, edgecolor='black')
    axes2[0, i].set_title(f'{col}', fontweight='bold')
    axes2[0, i].set_xlabel(col)
    axes2[0, i].set_ylabel('Count')
    axes2[0, i].tick_params(axis='x', rotation=45)
    axes2[0, i].grid(True, alpha=0.3)
    axes2[0, i].set_xticks(range(len(value_counts)))
    axes2[0, i].set_xticklabels(value_counts.index, rotation=45, ha='right')
    
    # Pie chart
    axes2[1, i].pie(value_counts.values, labels=value_counts.index, 
                   autopct='%1.1f%%', startangle=90)
    axes2[1, i].set_title(f'{col}', fontweight='bold')

plt.tight_layout()
plt.show()

# FIGURE 3: Last 3 columns (6 charts)
fig3, axes3 = plt.subplots(2, 3, figsize=(18, 12))
#fig3.suptitle('Categorical Variables - Group 3 (Bar + Pie Charts)', fontsize=16, fontweight='bold')

for i, col in enumerate(group3):
    value_counts = df_clean[col].value_counts().head(8)
    
    # Bar chart
    axes3[0, i].bar(range(len(value_counts)), value_counts.values, 
                   color='lightgreen', alpha=0.7, edgecolor='black')
    axes3[0, i].set_title(f'{col}', fontweight='bold')
    axes3[0, i].set_xlabel(col)
    axes3[0, i].set_ylabel('Count')
    axes3[0, i].tick_params(axis='x', rotation=45)
    axes3[0, i].grid(True, alpha=0.3)
    axes3[0, i].set_xticks(range(len(value_counts)))
    axes3[0, i].set_xticklabels(value_counts.index, rotation=45, ha='right')
    
    # Pie chart
    axes3[1, i].pie(value_counts.values, labels=value_counts.index, 
                   autopct='%1.1f%%', startangle=90)
    axes3[1, i].set_title(f'{col}', fontweight='bold')

plt.tight_layout()
plt.show()


## Correlation Matrix

In [ ]:
from dython.nominal import associations
import seaborn as sns

# matriz de correlacion incluyendo variables categoricas
def matriz_correlacion_categoricas(df):
    associations_df = associations(df_clean, nominal_columns=categorical_columns, plot=False)
    corr_matrix = associations_df['corr']
    plt.figure(figsize=(15, 10))
    plt.title('Correlation Matrix Including Categorical Variables', fontsize=16)
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True, linewidths=.5)
    plt.show()

matriz_correlacion_categoricas(df_clean)

## Random Forest model training


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

X = df_clean.drop(columns=['price'])
y = df_clean['price']

cat_cols = X.select_dtypes(include=['object']).columns
num_cols = X.select_dtypes(include=['number']).columns

#First we need to encode categorical columns so the model can be trained
preprocess = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('num', 'passthrough', num_cols)
    ]
)

# Reduce overfitting by limiting tree depth, minimum samples per leaf and minimum samples to split a node
# parameters were optimized with search_grid, but took 23 minutes to execute, taht is why has not been included here
model = Pipeline(steps=[
    ('prep', preprocess),
    ('reg', RandomForestRegressor(random_state=1, n_estimators=100, max_depth=7, min_samples_leaf=10, min_samples_split=2))
])

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
model.fit(train_X, train_y)

# Predict with all validation observations
val_predictions = model.predict(val_X)
train_predictions = model.predict(train_X)

from IPython.display import display, Markdown

# Display predictions and actual prices in parallel columns for validation and training sets using HTML tables (no tabulate dependency)
def make_html_table(df, title):
    html = f'<b>{title}</b><br><table><tr>'
    for col in df.columns:
        html += f'<th>{col}</th>'
    html += '</tr>'
    for _, row in df.iterrows():
        html += '<tr>' + ''.join(f'<td>{v}</td>' for v in row) + '</tr>'
    html += '</table>'
    return html

val_df = pd.DataFrame({
    'Validation Prediction': val_predictions[:5],
    'Validation Actual': list(val_y[:5])
})
train_df = pd.DataFrame({
    'Train Prediction': train_predictions[:5],
    'Train Actual': list(train_y[:5])
})

output = []
output.append("<br>" + "="*60)
output.append("RANDOM FOREST REGRESSOR RESULTS")
output.append("="*60)
output.append(make_html_table(val_df, "Validation Set (Top 5)"))
output.append(make_html_table(train_df, "Training Set (Top 5)"))
output.append("-"*60)
val_mae = mean_absolute_error(val_y, val_predictions)
train_mae = mean_absolute_error(train_y, train_predictions)
val_r2 = r2_score(val_y, val_predictions)
train_r2 = r2_score(train_y, train_predictions)
output.append(f"Validation MAE: {val_mae:.2f}")
output.append("<br>")
output.append(f"Training MAE: {train_mae:.2f}")
output.append("<br>")
output.append(f"Validation R2: {val_r2:.3f}")
output.append("<br>")
output.append(f"Training R2: {train_r2:.3f}")
display(Markdown('  \n'.join(output)))

## Train model with whole dataset and create csv file with predictions to feed it into the frontend

In [ ]:
# Train model on the entire df_clean and generate predictions for the frontend
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

# Prepare features and target
X_full = df_clean.drop(columns=['price'])
y_full = df_clean['price']

cat_cols = X_full.select_dtypes(include=['object']).columns
num_cols = X_full.select_dtypes(include=['number']).columns

preprocess = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols),
    ('num', 'passthrough', num_cols)
], remainder='drop')

model = Pipeline([
    ('prep', preprocess),
    ('reg', RandomForestRegressor(random_state=1, n_estimators=100, max_depth=7, min_samples_leaf=10, min_samples_split=2))
])

model.fit(X_full, y_full)

# Predict prices for the whole dataset
predicted_prices = model.predict(X_full)

# Create a new DataFrame with all features and predicted price
df_pred = X_full.copy()
df_pred['predicted_price'] = predicted_prices

# Save to CSV for frontend use
df_pred.to_csv('../datasets/predicted_prices_for_frontend.csv', index=False)
df_pred.head()